# Find the sets of 5 words with no char in common

> As each word in 5 chars length, the 5 words should span 25 characters of the alphabet

- Date : 22-Feb-2026
- Run Time < 1.5 Min (Local M4) (3 Mins on Colab)


In [1]:
import requests
from tqdm.notebook import tqdm
from collections import defaultdict
from string import ascii_lowercase

## Get word list

In [2]:
def get_words():
    # Source: https://gist.github.com/subhrm/5362767af06597bd1e216c59b760f6cb
    url="https://gist.githubusercontent.com/subhrm/5362767af06597bd1e216c59b760f6cb/raw/6bfa15d263d6d5b63840a8e5b64e04b382fdb079/valid-wordle-words.txt"

    # Download word list
    resp = requests.get(url)
    print(f"{resp.status_code=}")
    resp.raise_for_status()

    # process the downloaded list
    word_list = list(set([w for w in resp.text.split("\n") if len(w)==5]))
    print(f"{len(word_list)=}")
    return word_list

word_list = get_words()

resp.status_code=200
len(word_list)=14855


In [3]:
# keep only words with unique chars
word_list = [w for w in word_list if len(set(w)) == 5]
print(f"{len(word_list)=}")

len(word_list)=9365


## Group anagrams

In [4]:
char_map = {c:i for i,c in enumerate(ascii_lowercase)}

def word_to_num(word):
    return sum( (1<<char_map[c]) for c in word)

# test
print(f"{word_to_num("abc")=}")
print(f"{word_to_num("ac")=}")
assert word_to_num("abc") == word_to_num("bac")


word_to_num("abc")=7
word_to_num("ac")=5


In [5]:
anagrams = defaultdict(list)
for w in word_list:
    num = word_to_num(w)
    anagrams[num].append(w)

unique_list = sorted(anagrams.keys())
print(len(unique_list))

5650


## Construct inveretd index for anagrams

In [6]:
# construct inverted negative index of anagrams
neg_one_map = defaultdict(list)
for i in range(25):
    for j in range(i+1, 26):
        key = (1<<i) | (1<<j)
        neg_one_map[key] = [ w for w in unique_list if w&key == 0]

neg_one_map_keys = sorted(neg_one_map.keys(), key=lambda x: len(neg_one_map[x]))
print(f"{len(neg_one_map_keys)=}")

len(neg_one_map_keys)=325


In [7]:
print([len(neg_one_map[k]) for k in neg_one_map_keys[:10]])
print([len(neg_one_map[k]) for k in neg_one_map_keys[-10:]])

[1774, 1921, 1926, 1959, 2059, 2072, 2136, 2194, 2260, 2286]
[5066, 5099, 5105, 5199, 5216, 5222, 5252, 5325, 5354, 5359]


## Counstruct 2 set

In [8]:
two_set = defaultdict(list)

for a in tqdm(unique_list):
    for key in neg_one_map_keys:
        if (a & key) == key:
            for b in neg_one_map[key]:
                if b>a and (a & b) == 0:
                    key = a|b
                    two_set[key].append((a,b))
            break
two_set_keys = sorted(two_set.keys())

  0%|          | 0/5650 [00:00<?, ?it/s]

In [9]:
print(f"{len(two_set_keys)=:,}")

len(two_set_keys)=593,720


In [10]:
samples = 2
for k in two_set_keys[:samples]:
    v = two_set[k]
    print(f"{k=} {len(v)=}")
    for a,b in v[:5]:
        print(f"{anagrams[a]} x {anagrams[b]}")
    print("="*40)

k=2559 len(v)=1
['badge', 'begad', 'debag'] x ['filch']
k=3007 len(v)=1
['chiba', 'biach'] x ['fjeld']


## Construct 3-set

In [11]:
three_set = defaultdict(list)

for a in tqdm(two_set_keys):
    for key in neg_one_map_keys:
        if (a & key) == key:
            for b in neg_one_map[key]:
                if (a & b) == 0:
                    key = a|b
                    three_set[key].append((a,b))
            break

print(f"{len(three_set)=:,}")

  0%|          | 0/593720 [00:00<?, ?it/s]

len(three_set)=1,107,285


In [12]:
three_set_keys = sorted(three_set.keys())
print(f"{len(three_set_keys)=:,}")

len(three_set_keys)=1,107,285


## Compute 5 set

In [13]:
from itertools import combinations
from functools import reduce

In [14]:
five_set = defaultdict(list)
super_key = (1<<26) - 1
with tqdm(total=len(three_set_keys)) as pbar:
    step = 500
    for idx,a in enumerate(three_set_keys):
        mask = super_key ^ a
        pos = [x for x in range(26) if (mask & (1<<x)) > 0]
        # assert len(pos) == 11
        comb = combinations(pos, 10)
        # assert len(comb) == 11
        for p in comb:
            req_key = reduce(lambda x,y: x|(1<<y), p, 0)
            if req_key > a and req_key in two_set:
                key = a | req_key
                five_set[key].append((a,req_key))

        if idx%step == 0:
            pbar.update(step)

print(f"{len(five_set)=:,}")

  0%|          | 0/1107285 [00:00<?, ?it/s]

len(five_set)=5


## Display Results

In [15]:
def expand_two_key(key):
    res = set()
    for a,b in two_set[key]:
        res.add(tuple(sorted((a,b))))

    return list(res)

def expand_three_key(key):
    res = set()
    for two_key,c in three_set[key]:
        for a,b in expand_two_key(two_key):
            res.add(tuple(sorted((a,b,c))))

    return list(res)

def print_five_set(k):
    res = set()
    for three_key , two_key in five_set[k]:
        threes_list = expand_three_key(three_key)
        twos_list = expand_two_key(two_key)

        for threes in threes_list:
            for twos in twos_list:
                res.add(tuple(sorted(threes + twos)))

    final_list = []
    for t in sorted(res):
        final_list.append(" | ".join(",".join(anagrams[k]) for k in t))
    return final_list

In [18]:
total = 0
for k in five_set.keys():
    results = print_five_set(k)
    total += len(results)
    for r in results:
        print(f"{r}")
print(f"Total number of results : {total}")

bling | treck | waqfs | jumpy | vozhd
pling | treck | waqfs | jumby | vozhd
brick | glent | waqfs | jumpy | vozhd
kreng | clipt | waqfs | jumby | vozhd
prick | glent | waqfs | jumby | vozhd
glent | bruck | waqfs | jimpy | vozhd
joked | crumb | waqfs | phynx | glitz
dreck | jumbo | waqfs | phynx | glitz
fjord | quack | wembs | phynx | glitz
brock | judge | waqfs | phynx | miltz
cromb | juked | waqfs | phynx | glitz
cromb | gived | waqfs | phynx | klutz
kempt | brung | waqfs | cylix,xylic | vozhd
blunk | waqfs | cimex | grypt | vozhd
clunk | waqfs | bemix | grypt | vozhd
bruck | goved | waqfs | phynx | miltz
bruck | moved | waqfs | phynx | glitz
bruck | veldt | waqfs | phynx | gizmo
cromb | jived | waqfs | phynx | klutz
fjord | chunk | vibex | gymps | waltz
fjord | gucks | vibex | nymph | waltz
jacks | whump | blonx | gyved | fritz
jumps | chawk,whack | blonx | gyved | fritz
Total number of results : 23
